In [7]:
#Setting Spark with MinIO
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import *
import os
from pyspark.sql import SparkSession
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
BUCKET_NAME = os.getenv("BUCKET_NAME")
POSTGRES_USER=os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD=os.getenv("POSTGRES_PASSWORD")
POSTGRES_ENDPOINT=os.getenv("POSTGRES_ENDPOINT")
DATA_ENPOINT=os.getenv("DATA_ENPOINT")

spark = SparkSession.builder \
    .appName("MinIOReader") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.jars", "/home/harshithts/jars/postgresql-42.7.3.jar") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .getOrCreate()


spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")



In [8]:
account_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-accounts/*.parquet")
profile_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-profiles/*.parquet")
merchant_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-merchants/*.parquet")



In [9]:


profile_df.write \
    .format("jdbc") \
    .option("url",DATA_ENPOINT) \
    .option("dbtable", "public.USERS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
account_df.write \
    .format("jdbc") \
    .option("url", DATA_ENPOINT) \
    .option("dbtable", "public.ACCOUNTS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
merchant_df.write \
    .format("jdbc") \
    .option("url", DATA_ENPOINT) \
    .option("dbtable", "public.MERCHANTS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()